In [1]:
class Person:
    def hello(arg: str = "default"):
        print(f"Hello, with `{arg}`")


class MyClass:

    @staticmethod
    def hello():
        print("Hello from MyClass.hello()")

    def instance_hello(self):
        print(f"Hello from MyClass.instance_hello(), with {self}")

    @classmethod
    def class_hello(cls):
        print(f"Hello from MyClass.class_hello(), with {cls}")


In [2]:
Person.hello()

Hello, with `default`


In [3]:
p = Person()
hex(id(p)), p.hello()


Hello, with `<__main__.Person object at 0x10ed44440>`


('0x10ed44440', None)

In [4]:
my_class = MyClass()
my_class.hello, my_class.hello()

Hello from MyClass.hello()


(<function __main__.MyClass.hello()>, None)

In [5]:
my_class.instance_hello, my_class.instance_hello()

Hello from MyClass.instance_hello(), with <__main__.MyClass object at 0x10f1dbcb0>


(<bound method MyClass.instance_hello of <__main__.MyClass object at 0x10f1dbcb0>>,
 None)

In [6]:
my_class.class_hello, my_class.class_hello()

Hello from MyClass.class_hello(), with <class '__main__.MyClass'>


(<bound method MyClass.class_hello of <class '__main__.MyClass'>>, None)

In [7]:
from datetime import datetime, timezone, timedelta

class Timer:
    tz = timezone.utc
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)

    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)


In [8]:
Timer.set_tz(-7, "MST")
Timer.tz


datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')

In [9]:
t1 = Timer()
t2 = Timer()

In [10]:
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST'),
 datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST'))

In [11]:
Timer.set_tz(2, "CET")
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(seconds=7200), 'CET'),
 datetime.timezone(datetime.timedelta(seconds=7200), 'CET'))

In [12]:
Timer.current_dt_utc(), t1.current_dt_utc(), t2.current_dt_utc()

(datetime.datetime(2024, 6, 19, 16, 18, 8, 1425, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 6, 19, 16, 18, 8, 1431, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 6, 19, 16, 18, 8, 1432, tzinfo=datetime.timezone.utc))

In [14]:
Timer.current_dt(), t1.current_dt(), t2.current_dt()

(datetime.datetime(2024, 6, 19, 18, 18, 34, 401032, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'CET')),
 datetime.datetime(2024, 6, 19, 18, 18, 34, 401039, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'CET')),
 datetime.datetime(2024, 6, 19, 18, 18, 34, 401040, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'CET')))

In [15]:
class TimerError(Exception):
    """A custom exception used for Timer class"""

In [25]:
class Timer:
    tz = timezone.utc
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)

    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)

    def start(self):
        self._time_start = self.current_dt_utc()
        self._time_end = None

    def stop(self):
        if not getattr(self, "_time_start", None) or not self._time_start:
            raise TimeError("Timer must be started before it can be stopped.")

        self._time_end = self.current_dt_utc()

    @property
    def start_time(self):
        """Timer `start_time` given in the Timer timezone.""" 
        if not getattr(self, "_time_start", None) or not self._time_start:
            raise TimeError("Timer has not been started.")
        return self._time_start.astimezone(self.tz)

    @property
    def end_time(self):
        """Timer `end_time` given in the Timer timezone."""
        if not getattr(self, "_time_end", None) or not self._time_end:
            raise TimeError("Timer has not been stopped.")
        return self._time_end.astimezone(self.tz)

    @property
    def elapsed(self):
        """Returns timedelta of how much time elapsed from time start"""
        if not getattr(self, "_time_start", None) or not self._time_start:
            raise TimeError("Timer must be started before an elapsed time can be calculated.")

        if not getattr(self, "_time_end", None) or not self._time_end:
            # you don't have to stop the Timer to check elapsed time
            elapsed_time = self.current_dt_uct() - self._start_time
        else:
            elapsed_time = self._time_end - self._time_start

        return elapsed_time


In [26]:
from time import sleep

t1 = Timer()
t1.start()
sleep(1)
t1.stop()
print(f"Start time {t1.start_time}")
print(f"End time {t1.end_time}")
print(f"Elapsed {t1.elapsed}")


Start time 2024-06-19 16:48:06.743129+00:00
End time 2024-06-19 16:48:07.746208+00:00
Elapsed 0:00:01.003079


In [27]:
t2 = Timer()
t2.start()
sleep(3)
t2.stop()
print(f"Start time {t2.start_time}")
print(f"End time {t2.end_time}")
print(f"Elapsed {t2.elapsed}")

Start time 2024-06-19 16:49:16.058764+00:00
End time 2024-06-19 16:49:19.063582+00:00
Elapsed 0:00:03.004818


In [28]:
Timer.set_tz(2, "CET")

In [30]:
# now it's with offset - every instance uses new timezone
print(f"Start time {t2.start_time}")
print(f"End time {t2.end_time}")
print(f"Elapsed {t2.elapsed}")

Start time 2024-06-19 18:49:16.058764+02:00
End time 2024-06-19 18:49:19.063582+02:00
Elapsed 0:00:03.004818


In [31]:
# now it's with offset - every instance uses new timezone
print(f"Start time {t1.start_time}")
print(f"End time {t1.end_time}")
print(f"Elapsed {t1.elapsed}")

Start time 2024-06-19 18:48:06.743129+02:00
End time 2024-06-19 18:48:07.746208+02:00
Elapsed 0:00:01.003079
